In [1]:
import requests
import bs4 

In [2]:
def get_directories(passed_html):
    """
    Passed a GitHub-Repo this function finds all of the child directories under that repo and returns a list of urls.
    """
    sub_directories = passed_html.find_all("a", class_="js-navigation-open Link--primary") # Finding all of the internal navigation links

    repositories = []   # A list to store the new repository urls.
    for sub_directory in sub_directories:   # Looping through the child directories as strings 
        for js_string in str(sub_directory).split('"'): # Retrieving the java-script strings 
            if js_string.startswith("/"):   # Finding the internal links by the first character 
                repositories.append("".join(["https://github.com", js_string])) # Adding the appropriate preface to the url.

    return repositories # Returning the list of internal directories

In [3]:
http_request = requests.get("https://github.com/SharmaLlama/ticktack/tree/main/src/data/datasets") # Getting the root directory
html_parser = bs4.BeautifulSoup(http_request.text, "html.parser")   # Passing the raw text of the directy as html for better manipulation 

In [4]:
data_sets = []  # Initialising a list to store the directories
for dir in get_directories(html_parser):    # Finding all the child directories of the root directory 
    data_set = requests.get(dir)    # Getting the raw http response from the child directory 
    passed_data = bs4.BeautifulSoup(data_set.text, "html.parser")   # Passing the child directory as html 

    data_sets.append(get_directories(passed_data))  # Storing the children's children in the data_sets list

In [9]:
for i, directory_data_sets in enumerate(data_sets): # Looping over the data sets
    for data_set_url in directory_data_sets:    # getting the urls of the data_sets
        if data_set_url.find("csv") == -1:  # Checking that the leaves of the directory have been reached 
            data_set = requests.get(data_set_url)   # Getting a response from the data set repo
            passed_data = bs4.BeautifulSoup(data_set.text, "html.parser")   # Passing the data set as
            data_sets[i] = get_directories(passed_data) # Updating the stored entry to make the list homogeneous

In [68]:
for data_urls in data_sets: # Loopig through the data sets in found by the program 
    for data_url in data_urls:  # Extracting the individual urls 
        data_set = requests.get(data_url)   # getting the http response for the data
        data_set = bs4.BeautifulSoup(data_set.text, "html.parser")  # Passing the data html 

        line_number_tags = data_set.find_all("td", class_="blob-num js-line-number")    # Finding the number of line in the html table that the data is displayed in 
        number_of_lines = int(line_number_tags[-1]["id"][1:]) - 1   # Getting the total number of lines as python would see it using the range function 

        data = {    # Storing the data as a python dictionary 
            "year": [], # A column to store the year 
            "d14c": [], # A column to store the carbon 14 concentrations 
            "sig_d14c": []  # A columns to store the error in the carbon 14 concentrations 
        }
        
        tabular_data = data_set.find_all("td")[:-1] # retrieving the tabular data 

        for line_number in range(number_of_lines * 4):  # Looping through each line of the table 
            column = line_number % 4    # Finding the column of the present namespace 
            if column == 1: # This represents the internal line markers 
                pass
            elif column == 2:   # This represents the year of the row 
                data["year"].append(tabular_data[line_number]) 
            elif column == 3:   # This represents the carbon content of the row 
                data["d14c"].append(tabular_data[line_number])
            elif column == 0:   # The represents the carbon error of the row 
                data["sig_d14c"].append(tabular_data[line_number])
